# Export All GT Panel Annotations
We use the images and train/val/test splits from the Bernie interviews.

In [15]:
from rekall import Interval, IntervalSet, IntervalSetMapping, Bounds3D
from rekall.predicates import *
from rekall.stdlib import ingest
import math
import urllib3, requests, os

In [4]:
urllib3.disable_warnings()

# Load Video Metadata

In [8]:
VIDEO_COLLECTION_BASEURL = "http://olimar.stanford.edu/hdd/tvnews-sandbox"
VIDEO_METADATA_FILENAME = "data/video_meta_sandbox.json"

In [9]:
req = requests.get(os.path.join(VIDEO_COLLECTION_BASEURL, VIDEO_METADATA_FILENAME), verify=False)
video_collection = req.json()

In [10]:
video_metadata = [
    VideoMetadata(v["path"], v["id"], v["fps"], int(v["num_frames"]), v["width"], v["height"])
    for v in video_collection
]

# Train, Val, Test Splits

In [1]:
train_set = [8220, 59398, 13141, 50164, 33004, 9901, 59122, 12837, 3769, 52075,
             10335, 57990, 45655, 57804, 24193, 3459, 37113, 2648, 8697, 57708,
             57592, 11003, 7262, 27410, 60186, 17983, 45472, 33387, 50561, 13556,
             16542, 40203, 53684, 11555, 37107, 51175, 23181, 49931, 24992, 14482,
             4611, 16879, 13827, 32472, 11792, 25463, 31378, 45645, 53355]
val_set = [38275, 42756, 52945, 34642, 19959, 37170, 55711, 45698, 20380, 3952,
           20450, 52749, 13927, 16215, 57384, 8859, 41725, 10323, 33541, 38420,
           23184, 19882, 17458, 34359]
test_set = [54377, 26386, 5281, 763, 9499, 24847, 13247, 29001, 9480, 9215, 27188,
            13058, 32996, 6185, 36755, 13993, 4143, 3730, 15916, 529, 11579, 48140,
            41480, 16693]

In [2]:
print(len(train_set + val_set + test_set))

97


# Load Ground Truth Annotations

In [5]:
PANELS_JSON = "data/panels.json"

In [6]:
def load_json(video_baseurl, json_path):
    req = requests.get(os.path.join(video_baseurl, json_path), verify=False)
    json_objs = req.json()
    ism = ingest.ism_from_iterable_with_schema_bounds3D(
        json_objs,
        ingest.getter_accessor,
        {
            'key': 'video_id',
            't1': 'start',
            't2': 'end'
        },
        with_payload = lambda item: item,
        progress = True
    )
    return ism

In [7]:
panels = load_json(VIDEO_COLLECTION_BASEURL, PANELS_JSON)

100%|██████████| 135/135 [00:00<00:00, 197155.65it/s]


## Downsample

In [13]:
vm_by_video = {
    video_id: [vm for vm in video_metadata if vm.id == video_id][0]
    for video_id in train_set + val_set + test_set
}

In [17]:
def frame_second_conversion(c, mode='f2s'):
    def second_to_frame(fps):
        def map_fn(intrvl):
            i2 = intrvl.copy()
            curr_bounds = intrvl['bounds'].copy()
            curr_bounds['t1'] = int(curr_bounds['t1']*fps)
            curr_bounds['t2'] = int(curr_bounds['t2']*fps)
            i2['bounds'] = curr_bounds
            return i2
        return map_fn
    
    def frame_to_second(fps):
        def map_fn(intrvl):
            i2 = intrvl.copy()
            curr_bounds = intrvl['bounds'].copy()
            curr_bounds['t1'] = int(curr_bounds['t1']/fps)
            curr_bounds['t2'] = int(curr_bounds['t2']/fps)
            i2['bounds'] = curr_bounds
            return i2
        return map_fn
    
    if mode=='f2s':
        fn = frame_to_second
    if mode=='s2f':
        fn = second_to_frame
    output = {}
    for vid, intervals in c.get_grouped_intervals().items():
        output[vid] = intervals.map(fn(vm_by_video[vid].fps))
    return IntervalSetMapping(output)

def frame_to_second_collection(c):
    return frame_second_conversion(c, 'f2s')

def second_to_frame_collection(c):
    return frame_second_conversion(c, 's2f')

In [18]:
interval = 30

In [19]:
segs_dict = {}
for video_id in train_set + val_set + test_set:
    video = vm_by_video[video_id]
    iset = IntervalSet([
        Interval(Bounds3D(i, i), video.fps)
        for i in range(0, video.num_frames) if (i % (
            math.floor(video.fps * 3) * (interval / 3)
        )) == 0
    ])
    segs_dict[video_id] = iset
    
segments = frame_to_second_collection(IntervalSetMapping(segs_dict)).dilate(interval / 2)

In [20]:
segments_all_negative = segments.map(
    lambda intrvl: Interval(intrvl['bounds'], 0)
)

In [21]:
def filter_by_id(ism, valid_ids):
    return IntervalSetMapping({
        vid: ism.get_grouped_intervals()[vid]
        for vid in list(ism.get_grouped_intervals().keys()) if vid in valid_ids
    })

In [22]:
panel_segments = segments.filter_against(
    panels, predicate = overlaps()
).map(
    lambda intrvl: Interval(intrvl['bounds'], 1)
)

panel_labels = segments_all_negative.minus(
    panel_segments
).union(panel_segments)

print(panel_segments.size())
print(panel_labels.size())

{13058: 11, 4611: 42, 42756: 25, 41480: 25, 9480: 13, 40203: 25, 48140: 7, 52749: 26, 11792: 7, 529: 29, 27410: 22, 26386: 21, 38420: 23, 60186: 14, 9499: 69, 8220: 17, 9215: 12, 11555: 42, 12837: 26, 57384: 7, 6185: 46, 4143: 25, 17458: 37, 37170: 11, 27188: 37, 16693: 11, 34359: 18, 17983: 38, 29001: 14, 45645: 16, 45655: 39, 2648: 36, 10335: 63, 13927: 25, 54377: 21, 33387: 47, 53355: 19, 3952: 44, 25463: 17, 24193: 13, 50561: 17, 38275: 19, 23184: 8, 31378: 13, 36755: 8, 8859: 23, 16542: 16, 45472: 44, 5281: 27, 24992: 19, 13993: 12, 19882: 30, 9901: 18, 52945: 21, 32472: 12, 20450: 12, 32996: 29, 16879: 13, 59122: 24, 37107: 26, 57592: 56, 37113: 108, 11003: 13, 41725: 7, 763: 28}
{13058: 124, 13827: 124, 4611: 123, 42756: 124, 59398: 124, 33541: 123, 41480: 123, 9480: 123, 40203: 364, 48140: 123, 52749: 245, 49931: 123, 24847: 123, 11792: 123, 529: 123, 27410: 124, 26386: 123, 38420: 124, 60186: 123, 9499: 366, 8220: 123, 9215: 123, 11555: 123, 12837: 124, 57384: 123, 6185: 123, 

# Export GT Labels

In [ ]:
BASEDIR = 'data'
os.makedirs(BASEDIR, exist_ok=True)
with open(os.path.join(BASEDIR, 'train.txt'), 'w') as f:
    for video_id in train_set:
        for i, intrvl in enumerate(interview_labels[video_id].get_intervals()):
            f.write('{} {} {}\n'.format(video_id, i, intrvl['payload']))
with open(os.path.join(BASEDIR, 'val.txt'), 'w') as f:
    for video_id in val_set:
        for i, intrvl in enumerate(interview_labels[video_id].get_intervals()):
            f.write('{} {} {}\n'.format(video_id, i, intrvl['payload']))
with open(os.path.join(BASEDIR, 'test.txt'), 'w') as f:
    for video_id in test_set:
        for i, intrvl in enumerate(interview_labels[video_id].get_intervals()):
            f.write('{} {} {}\n'.format(video_id, i, intrvl['payload']))